Purpose of this notebook is for a selected region (N1 data) - get the best per fold parameters DT, ADA , RF <br>
Use those parameters for train, validation, and test <br>
Use the train, validation, test scores to understand what is going on with training - expect that the Random Forest is underfitting

In [2]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupShuffleSplit

from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupKFold

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts


#TS Fresh Parameter Settings
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.feature_extraction import MinimalFCParameters
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.feature_extraction import extract_features

import matplotlib.patches as mpatches

#### Load files

In [3]:
files_list = ['/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Baseline_Extensions/Fitting Hyperparam Test/hyperparameter_tuning_results/Ada_N1_Minimal_beta_expt1_5_fold_results_df.pkl',
 '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Baseline_Extensions/Fitting Hyperparam Test/hyperparameter_tuning_results/DT_N1_Minimal_beta_expt1_5_fold_results_df.pkl',
 '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Baseline_Extensions/Fitting Hyperparam Test/hyperparameter_tuning_results/RF_N1_Minimal_beta_expt1_5_fold_results_df.pkl',
 '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Baseline_Extensions/Fitting Hyperparam Test/hyperparameter_tuning_results/SVC_N1_Minimal_beta_expt1_5_fold_results_df.pkl']

In [4]:
## Helper function to return accuracy dataframe for each file
def return_acc_df(results_df):
    '''
    Input : Dataframe containing all raw results from hyperparameter tuning
    Output: Dataframe with accuracy per region for that classifier
    '''
    results_df = results_df.drop(columns = [col for col in results_df.columns if 'params' in col] +  [col for col in results_df.columns if 'results' in col] )
    res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
    acc_df = res_df[[col for col in res_df.columns if 'acc' in col]]
    mean_acc_df = pd.DataFrame( acc_df.mean(axis=0) ).T
    
    return mean_acc_df

### Select region to assess further

In [5]:
region = 'Parietal' # DT > Ada > RF > SVC 

In [6]:
all_results_df = joblib.load(files_list[0])
region_results_df = all_results_df[ ['random_state' ,  'y_true'] + [col for col in all_results_df if 'Parietal' in col]  ]

In [7]:
region_results_df

,random_state,y_true,Parietal_Ada_y_preds,Parietal_Ada_best_clf_params,Parietal_Ada_cv_results,Left Parietal_Ada_y_preds,Left Parietal_Ada_best_clf_params,Left Parietal_Ada_cv_results,Right Parietal_Ada_y_preds,Right Parietal_Ada_best_clf_params,Right Parietal_Ada_cv_results
0,1,"[[0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 0, 1, 1, 1, ...","[[0, 0, 0, 0, 0, 0, 1, 0], [1, 0, 1, 1, 1, 1, ...","[{'learning_rate': 0.01, 'n_estimators': 2}, {...",mean_fit_time std_fit_time mean_score_ti...,"[[0, 0, 1, 1, 0, 0, 1, 1], [0, 0, 0, 1, 1, 0, ...","[{'learning_rate': 0.05, 'n_estimators': 50}, ...",mean_fit_time std_fit_time mean_score_ti...,"[[0, 0, 1, 1, 0, 0, 1, 1], [1, 0, 0, 1, 1, 0, ...","[{'learning_rate': 0.2, 'n_estimators': 20}, {...",mean_fit_time std_fit_time mean_score_ti...
1,2,"[[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 1, ...","[[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 1, ...","[{'learning_rate': 0.05, 'n_estimators': 2}, {...",mean_fit_time std_fit_time mean_score_ti...,"[[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 1, ...","[{'learning_rate': 0.2, 'n_estimators': 3}, {'...",mean_fit_time std_fit_time mean_score_ti...,"[[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 1, ...","[{'learning_rate': 10.0, 'n_estimators': 3}, {...",mean_fit_time std_fit_time mean_score_ti...


In [8]:
# Function to unique a list of dictionaries
def u_dicts(dicts_list):
    '''return set of unique dictionaries from input list'''
    # Create a set to store unique dictionaries
    unique_dicts = set()
    # Iterate through the list and add each dictionary as a tuple to the set
    for d in list(dicts_list):
        unique_dicts.add(tuple(d.items()))
    # Convert the unique tuples back to dictionaries
    unique_dicts = [dict(items) for items in unique_dicts]
    return unique_dicts

In [13]:
all_results_df = joblib.load(file)

In [14]:
all_results_df

,random_state,y_true,Prefrontal_SVC_y_preds,Prefrontal_SVC_best_clf_params,Prefrontal_SVC_cv_results,Frontal_SVC_y_preds,Frontal_SVC_best_clf_params,Frontal_SVC_cv_results,Left Frontal_SVC_y_preds,Left Frontal_SVC_best_clf_params,...,Parietal_SVC_cv_results,Left Parietal_SVC_y_preds,Left Parietal_SVC_best_clf_params,Left Parietal_SVC_cv_results,Right Parietal_SVC_y_preds,Right Parietal_SVC_best_clf_params,Right Parietal_SVC_cv_results,Occipital_SVC_y_preds,Occipital_SVC_best_clf_params,Occipital_SVC_cv_results
0,1,"[[0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 0, 1, 1, 1, ...","[[1, 0, 0, 1, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0, ...","[{'C': 0.1, 'gamma': 1, 'kernel': 'linear'}, {...",mean_fit_time std_fit_time mean_score_ti...,"[[1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[{'C': 0.1, 'gamma': 1, 'kernel': 'linear'}, {...",mean_fit_time std_fit_time mean_score_ti...,"[[0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[{'C': 0.1, 'gamma': 1, 'kernel': 'linear'}, {...",...,mean_fit_time std_fit_time mean_score_ti...,"[[1, 0, 1, 1, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, ...","[{'C': 0.1, 'gamma': 1, 'kernel': 'linear'}, {...",mean_fit_time std_fit_time mean_score_ti...,"[[0, 0, 1, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[{'C': 0.1, 'gamma': 1, 'kernel': 'linear'}, {...",mean_fit_time std_fit_time mean_score_ti...,"[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[{'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}, {'C'...",mean_fit_time std_fit_time mean_score_ti...
1,2,"[[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 1, ...","[[0, 0, 1, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0, ...","[{'C': 0.1, 'gamma': 1, 'kernel': 'linear'}, {...",mean_fit_time std_fit_time mean_score_ti...,"[[1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, ...","[{'C': 0.1, 'gamma': 1, 'kernel': 'linear'}, {...",mean_fit_time std_fit_time mean_score_ti...,"[[0, 1, 1, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[{'C': 0.1, 'gamma': 1, 'kernel': 'linear'}, {...",...,mean_fit_time std_fit_time mean_score_ti...,"[[1, 1, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[{'C': 0.1, 'gamma': 1, 'kernel': 'linear'}, {...",mean_fit_time std_fit_time mean_score_ti...,"[[0, 1, 1, 1, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, ...","[{'C': 0.1, 'gamma': 1, 'kernel': 'linear'}, {...",mean_fit_time std_fit_time mean_score_ti...,"[[0, 0, 1, 1, 1, 1, 0, 1], [0, 0, 0, 0, 0, 0, ...","[{'C': 0.1, 'gamma': 1, 'kernel': 'linear'}, {...",mean_fit_time std_fit_time mean_score_ti...


### First try for one random state, all models

In [15]:
best_params_dfs = []
for file, model in zip(files_list, ['Ada', 'DT' , 'RF' , 'SVC']):
    df_list = [model]
    for r_state_row in [0,1]:
        all_results_df = joblib.load(file)
        region_results_df = all_results_df[ ['random_state' ,  'y_true'] + [col for col in all_results_df if 'Occipital' in col]  ]
            
        random_state_1_df = pd.DataFrame(region_results_df.iloc[r_state_row,:] ).T
        cv_results_df =  random_state_1_df['Occipital_' + model + '_cv_results'].values[0]
       
        acc_scores = []
        for y_true, y_pred in zip(list(random_state_1_df['y_true'].values[0]), list(random_state_1_df['Occipital_' + model + '_y_preds'].values[0]) ):
            acc = accuracy_score(y_true, y_pred)
            acc_scores.append(acc)
        acc_scores
        
        best_per_fold = []
        for j, params in enumerate(random_state_1_df['Occipital_' + model + '_best_clf_params'].values[0]):
            j = j+1
            best_per_fold.append(cv_results_df[ (cv_results_df['params'] == params) & (cv_results_df['fold'] == j) ])
        
        best_per_fold_df = pd.concat(best_per_fold)    
        best_per_fold_df = best_per_fold_df.iloc[: , 4:]
        best_per_fold_df = best_per_fold_df.drop(columns = [col for col in best_per_fold_df if 'split' in col])
        df_list.append(best_per_fold_df)
    best_params_dfs.append(df_list)

In [25]:
Ada = pd.concat(best_params_dfs[2][1:])
Ada.iloc[:,4].values

array([{'criterion': 'entropy', 'max_depth': 5, 'max_features': None, 'min_samples_split': 5, 'n_estimators': 1},
       {'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 1},
       {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 2},
       {'criterion': 'gini', 'max_depth': 2, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 1},
       {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 1},
       {'criterion': 'entropy', 'max_depth': 2, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 1},
       {'criterion': 'gini', 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 10},
       {'criterion': 'entropy', 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 3},
       {'criterion': 'gini', 'max_depth': 2, 'max_features': None, 'min_samples_sp

In [26]:
for df_list in best_params_dfs:
    print(df_list[0])
    display(pd.concat(df_list[1:]))

Ada


,param_learning_rate,param_n_estimators,params,mean_test_score,std_test_score,rank_test_score,fold
19,0.1,10,"{'learning_rate': 0.1, 'n_estimators': 10}",0.823810,0.166053,1,1
6,0.01,50,"{'learning_rate': 0.01, 'n_estimators': 50}",0.785714,0.150585,1,2
28,0.2,20,"{'learning_rate': 0.2, 'n_estimators': 20}",0.847619,0.091101,1,3
10,0.05,5,"{'learning_rate': 0.05, 'n_estimators': 5}",0.666667,0.290437,1,4
37,0.4,40,"{'learning_rate': 0.4, 'n_estimators': 40}",0.876190,0.110040,1,5
22,0.1,50,"{'learning_rate': 0.1, 'n_estimators': 50}",0.842857,0.139321,1,1
12,0.05,20,"{'learning_rate': 0.05, 'n_estimators': 20}",0.847619,0.091101,1,2
0,0.01,2,"{'learning_rate': 0.01, 'n_estimators': 2}",0.780952,0.075892,1,3
1,0.01,3,"{'learning_rate': 0.01, 'n_estimators': 3}",0.819048,0.174054,1,4
58,10.0,5,"{'learning_rate': 10.0, 'n_estimators': 5}",0.852381,0.156782,1,5


DT


,param_criterion,param_max_depth,param_min_samples_leaf,params,mean_test_score,std_test_score,rank_test_score,fold
5,gini,2,1,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.842857,0.139321,1,1
45,entropy,None,1,"{'criterion': 'entropy', 'max_depth': None, 'm...",0.804762,0.195412,1,2
11,gini,3,2,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",0.904762,0.078246,1,3
31,entropy,2,2,"{'criterion': 'entropy', 'max_depth': 2, 'min_...",0.638095,0.358695,1,4
11,gini,3,2,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",0.909524,0.117031,1,5
2,gini,1,3,"{'criterion': 'gini', 'max_depth': 1, 'min_sam...",0.819048,0.138505,1,1
23,gini,None,5,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.776190,0.089847,1,2
11,gini,3,2,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",0.876190,0.110040,1,3
43,entropy,5,5,"{'criterion': 'entropy', 'max_depth': 5, 'min_...",0.909524,0.074383,1,4
41,entropy,5,2,"{'criterion': 'entropy', 'max_depth': 5, 'min_...",0.852381,0.128130,1,5


RF


,param_criterion,param_max_depth,param_max_features,param_n_estimators,params,mean_test_score,std_test_score,rank_test_score,fold
413,entropy,5,None,1,"{'criterion': 'entropy', 'max_depth': 5, 'max_...",0.795238,0.192607,1,1
161,gini,5,sqrt,1,"{'criterion': 'gini', 'max_depth': 5, 'max_fea...",0.842857,0.139321,1,2
442,entropy,5,sqrt,2,"{'criterion': 'entropy', 'max_depth': 5, 'max_...",0.933333,0.133333,1,3
49,gini,2,sqrt,1,"{'criterion': 'gini', 'max_depth': 2, 'max_fea...",0.809524,0.188682,1,4
441,entropy,5,sqrt,1,"{'criterion': 'entropy', 'max_depth': 5, 'max_...",0.885714,0.166599,1,5
329,entropy,2,sqrt,1,"{'criterion': 'entropy', 'max_depth': 2, 'max_...",0.847619,0.091101,1,1
94,gini,3,sqrt,10,"{'criterion': 'gini', 'max_depth': 3, 'max_fea...",0.814286,0.117031,1,2
380,entropy,3,sqrt,3,"{'criterion': 'entropy', 'max_depth': 3, 'max_...",0.880952,0.108588,1,3
14,gini,2,None,1,"{'criterion': 'gini', 'max_depth': 2, 'max_fea...",0.909524,0.117031,1,4
21,gini,2,None,1,"{'criterion': 'gini', 'max_depth': 2, 'max_fea...",0.852381,0.128130,1,5


SVC


,param_C,param_gamma,param_kernel,params,mean_test_score,std_test_score,rank_test_score,fold
1,0.1,1,rbf,"{'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}",0.380952,0.294700,1,1
1,0.1,1,rbf,"{'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}",0.680952,0.187839,1,2
1,0.1,1,rbf,"{'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}",0.652381,0.197260,1,3
1,0.1,1,rbf,"{'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}",0.609524,0.298370,1,4
0,0.1,1,linear,"{'C': 0.1, 'gamma': 1, 'kernel': 'linear'}",0.438095,0.151785,1,5
0,0.1,1,linear,"{'C': 0.1, 'gamma': 1, 'kernel': 'linear'}",0.457143,0.190119,1,1
1,0.1,1,rbf,"{'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}",0.657143,0.046657,1,2
1,0.1,1,rbf,"{'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}",0.628571,0.146462,1,3
1,0.1,1,rbf,"{'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}",0.647619,0.201694,1,4
1,0.1,1,rbf,"{'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}",0.623810,0.277828,1,5
